In [1]:
import geopandas as gpd

## Open GIS Data

In [14]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_gdf = gpd.read_file(file_path)
states_gdf = states_gdf[["STUSPS", "geometry"]]

## Open Macy's Data

In [20]:
macys_gdf = gpd.read_file("data/macys_per_state.gpkg")
macys_gdf = macys_gdf.to_crs(9311)
macys_gdf["macys"] = macys_gdf["stores"].astype(int)
macys_gdf = macys_gdf[["STUSPS", "macys"]]

## Get Nordstroms Data

In [34]:
nordstrom_gdf = gpd.read_file("data/nordstrom_per_state.gpkg")
nordstrom_gdf = nordstrom_gdf.to_crs(9311)
nordstrom_gdf["nordstroms"] = nordstrom_gdf["NORDSTROMS"].astype(int) + nordstrom_gdf[
    "NORDSTROM_RACKS"
].astype(int)
nordstrom_gdf = nordstrom_gdf[["STUSPS", "nordstroms"]]

## Merge Data

In [35]:
states_with_macys_gdf = states_gdf.merge(macys_gdf, how="left", on="STUSPS")

In [36]:
macys_vs_nordstrom_gdf = states_with_macys_gdf.merge(
    nordstrom_gdf, how="left", on="STUSPS"
)

In [37]:
macys_vs_nordstrom_gdf = macys_vs_nordstrom_gdf.fillna(0)
macys_vs_nordstrom_gdf["nordstroms"] = macys_vs_nordstrom_gdf["nordstroms"].astype(int)
macys_vs_nordstrom_gdf["macys"] = macys_vs_nordstrom_gdf["macys"].astype(int)

In [42]:
macys_vs_nordstrom_gdf["macys_vs_nordstrom_label"] = (
    macys_vs_nordstrom_gdf["macys"].astype(str)
    + ":"
    + macys_vs_nordstrom_gdf["nordstroms"].astype(str)
)
macys_vs_nordstrom_gdf["macys_vs_nordstrom"] = (
    macys_vs_nordstrom_gdf["macys"] - macys_vs_nordstrom_gdf["nordstroms"]
)

In [43]:
macys_vs_nordstrom_gdf = macys_vs_nordstrom_gdf.to_crs(9311)
macys_vs_nordstrom_gdf.to_file(f"data/macys_vs_nordstrom.gpkg")